# 序列建模前沿技术综述（2025）

## 1. RNN（循环神经网络）
### 核心思想
RNN通过时间步的递归结构处理序列数据，利用隐藏状态 $h_t$ 捕捉历史信息

$$h_t = \sigma(W_h h_{t-1} + W_x x_t)$$

其中：
- $h_t$: 当前隐藏状态
- $x_t$: 时间步$t$的输入
- $\sigma$: 激活函数（如Tanh/ReLU）

### 问题与改进
**梯度问题**：
- 梯度消失/爆炸 RNN在长序列中难以学习长期依赖。
- （LSTM/GRU解决）引入门控机制（输入门、遗忘门、输出门），控制信息流动：
  
**LSTM门控机制**：
$$
\begin{aligned}
f_t &= \sigma(W_f [h_{t-1}, x_t]) \\
i_t &= \sigma(W_i [h_{t-1}, x_t]) \\
C_t &= f_t \odot C_{t-1} + i_t \odot \tanh(W_C [h_{t-1}, x_t]) \\
o_t &= \sigma(W_o [h_{t-1}, x_t]) \\
h_t &= o_t \odot \tanh(C_t)
\end{aligned}
$$



## 2. Transformer

Transformer通过 自注意力机制（Self-Attention）替代RNN的递归结构，直接建模全局依赖：

### 自注意力机制
$$
\text{Attention}(Q,K,V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

其中：
- $Q = XW^Q$, $K = XW^K$, $V = XW^V$
- $d_k$: 缩放因子（防止点积过大）

### 多头注意力

将输入拆分为多个头（Head），并行计算不同子空间的信息：

$$
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1,...,\text{head}_h)W^O
$$
$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

### 位置编码

通过正弦/余弦函数或学习嵌入向量注入序列位置信息：

$$
\begin{aligned}
PE_{(pos,2i)} &= \sin\left(\frac{pos}{10000^{2i/d}}\right) \\
PE_{(pos,2i+1)} &= \cos\left(\frac{pos}{10000^{2i/d}}\right)
\end{aligned}
$$

### 优势

- 并行化：适合GPU/TPU加速。
- 长距离依赖：直接建模全局关系。


## 3. Attention机制演进
### 传统Attention vs. Self-Attention
- 传统Attention：在序列到序列模型中，通过注意力权重对齐源序列和目标序列。
- Self-Attention：序列内部元素间的相关性建模（如Transformer）。

### 线性注意力
为解决自注意力的高计算复杂度O(n^2)，提出线性注意力：

$$
\text{LinearAttention}(Q,K,V) = \phi(Q)^T \phi(K)V
$$
- 通过核函数$\phi$将点积转换为低维空间操作降低计算复杂度至$O(n)$

### 改进
- RetNet：引入 Retention Mechanism，通过反射（Reflection）机制逐步遗忘旧信息：
$$
h t​=α t​⋅h t−1​+(1−α t​)⋅LinearAttention(x t​)
$$
- Gated Retention：增加门控控制遗忘速率。


## 4. RNN改进方案
### DeltaNet

将RNN与线性注意力结合，通过差分方程建模序列：

$$
h_t = h_{t-1} + \Delta h_t, \quad \Delta h_t = \text{LinearAttention}(x_t)
$$

利用线性注意力的低复杂度特性，提升RNN的效率。

### Linger（SSM模型）

通过 状态空间模型（SSM） 改进RNN，捕捉长序列依赖：

$$
\begin{aligned}
h_t &= A h_{t-1} + B x_t \\
y_t &= C h_t + D x_t
\end{aligned}
$$

其中 A,B,C,D 是可学习参数，SSM的计算复杂度为 O(n)。



## 5. Mamba（选择性SSM）

核心思想
Mamba通过 选择性状态空间模型（Selective SSM） 处理长序列，结合线性注意力和RNN的优势：

### 核心方程
$$
\Delta h_t = A(\theta) \cdot h_{t-1} + B(\theta) \cdot x_t
$$
- $A(\theta), B(\theta)$: 输入依赖的动态参数

是输入依赖的可学习参数，通过门控选择性更新状态。

### 关键创新
- 选择性机制：根据输入动态调整状态转移矩阵，提升灵活性。
- 高效计算：利用线性代数优化，将复杂度从 $O(n^2)$ 降至 $O(n)$
- 对比Transformer
    - 长序列优势：Mamba在超长序列（如100k tokens）上显著优于Transformer。
    - 硬件适配性：更适合GPU/TPU的并行计算。



## 6. 最新研究趋势
| 方向              | 代表技术               | 核心改进                 |
|-------------------|------------------------|--------------------------|
| 模型压缩         | LoLCATs              | 低秩注意力权重迁移       |
| 记忆增强         | Titans               | 动态记忆参数学习         |
| Mamba扩展        | MambaOut            | 视觉任务适配             |
